# logger

> Logging for RLM iterations - structured JSON-lines and Rich console output

In [ ]:
#| default_exp logger

## Overview

This module provides two complementary logging mechanisms for RLM, following the pattern established in [rlmpaper](https://github.com/alexzhang13/rlm):

1. **RLMLogger**: Structured JSON-lines logging for analysis and debugging
2. **VerbosePrinter**: Beautiful console output using Rich for real-time visibility

### Design Principles

- **Optional**: Both logger and verbose mode are opt-in
- **Non-invasive**: Minimal coupling to core RLM logic
- **Dual output**: Machine-readable JSONL + human-readable Rich output
- **Aligned with rlmpaper**: Same concepts, adapted for claudette backend

### References

- [rlmpaper logging](https://github.com/alexzhang13/rlm/tree/main/rlm/logger) - Reference implementation
- [Rich library](https://github.com/Textualize/rich) - Beautiful terminal formatting

## Imports

In [ ]:
import json
import os
import uuid
from datetime import datetime
from pathlib import Path
from typing import Any
from rlm._rlmpaper_compat import RLMIteration, CodeBlock, REPLResult

In [ ]:
#| export


## RLMLogger - Structured JSON-Lines Logging

Writes iteration data to JSON-lines files for post-hoc analysis.

Each line is a JSON object representing either:
- Metadata (first line): Configuration and setup
- Iteration: Complete iteration with prompt, response, code blocks, results

In [ ]:
#| export
class RLMLogger:
    """Logger that writes RLM iteration data to JSON-lines files.
    
    Creates timestamped JSONL files with complete iteration history.
    Following rlmpaper's logging pattern.
    
    Example:
        logger = RLMLogger(log_dir='./logs')
        logger.log_metadata({'query': 'What is X?', 'max_iters': 5})
        logger.log(iteration)
    """
    
    def __init__(self, log_dir: str | Path, file_name: str = 'rlm'):
        """Initialize logger with output directory.
        
        Args:
            log_dir: Directory for log files (created if doesn't exist)
            file_name: Base name for log files (default: 'rlm')
        """
        self.log_dir = Path(log_dir)
        self.log_dir.mkdir(parents=True, exist_ok=True)
        
        timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
        run_id = str(uuid.uuid4())[:8]
        self.log_file_path = self.log_dir / f"{file_name}_{timestamp}_{run_id}.jsonl"
        
        self._iteration_count = 0
        self._metadata_logged = False
    
    def log_metadata(self, metadata: dict):
        """Log RLM metadata as the first entry in the file.
        
        Args:
            metadata: Dict with query, context, max_iters, etc.
        """
        if self._metadata_logged:
            return
        
        entry = {
            'type': 'metadata',
            'timestamp': datetime.now().isoformat(),
            **metadata
        }
        
        with open(self.log_file_path, 'a') as f:
            json.dump(entry, f)
            f.write('\n')
        
        self._metadata_logged = True
    
    def log(self, iteration: RLMIteration, iteration_num: int):
        """Log an iteration to the file.
        
        Args:
            iteration: RLMIteration object to log
            iteration_num: Iteration number (1-indexed)
        """
        self._iteration_count += 1
        
        # Convert iteration to dict
        entry = {
            'type': 'iteration',
            'iteration': iteration_num,
            'timestamp': datetime.now().isoformat(),
            'prompt': iteration.prompt,
            'response': iteration.response,
            'code_blocks': [
                {
                    'code': block.code,
                    'result': {
                        'stdout': block.result.stdout,
                        'stderr': block.result.stderr,
                        'execution_time': block.result.execution_time
                    }
                }
                for block in iteration.code_blocks
            ]
        }
        
        with open(self.log_file_path, 'a') as f:
            json.dump(entry, f)
            f.write('\n')
    
    @property
    def iteration_count(self) -> int:
        """Number of iterations logged."""
        return self._iteration_count

Test RLMLogger:

In [ ]:
import tempfile


In [ ]:
# Create logger in temp directory
with tempfile.TemporaryDirectory() as tmpdir:
    logger = RLMLogger(tmpdir, file_name='test')
    
    # Log metadata
    logger.log_metadata({'query': 'Test query', 'max_iters': 5})
    
    # Create mock iteration
    result = REPLResult(stdout='Hello', stderr='', locals={}, execution_time=0.1)
    block = CodeBlock(code='print("Hello")', result=result)
    iteration = RLMIteration(prompt='Test prompt', response='Test response', code_blocks=[block])
    
    # Log iteration
    logger.log(iteration, 1)
    
    # Verify file exists and has content
    assert logger.log_file_path.exists()
    
    # Read and parse
    lines = logger.log_file_path.read_text().strip().split('\n')
    assert len(lines) == 2  # metadata + 1 iteration
    
    metadata_entry = json.loads(lines[0])
    assert metadata_entry['type'] == 'metadata'
    assert metadata_entry['query'] == 'Test query'
    
    iteration_entry = json.loads(lines[1])
    assert iteration_entry['type'] == 'iteration'
    assert iteration_entry['iteration'] == 1
    assert iteration_entry['response'] == 'Test response'
    assert len(iteration_entry['code_blocks']) == 1
    
    print(f"✓ RLMLogger works: {logger.log_file_path.name}")
    print(f"  Logged {logger.iteration_count} iteration(s)")

## VerbosePrinter - Rich Console Output

Beautiful, human-readable terminal output for debugging.

Simplified from rlmpaper's version - focuses on core visibility without heavy styling.

In [ ]:
    from rich.console import Console
    from rich.panel import Panel
    from rich.syntax import Syntax
    from rich.text import Text
            from rich.rule import Rule
            from rich.rule import Rule

In [ ]:
#| export
try:
    RICH_AVAILABLE = True
except ImportError:
    RICH_AVAILABLE = False


class VerbosePrinter:
    """Console printer for RLM verbose output using Rich.
    
    Provides real-time visibility into RLM execution with beautiful formatting.
    Falls back to simple print if Rich is not installed.
    
    Example:
        verbose = VerbosePrinter(enabled=True)
        verbose.print_header(query='What is X?', max_iters=5)
        verbose.print_iteration(iteration, 1)
    """
    
    def __init__(self, enabled: bool = True):
        """Initialize printer.
        
        Args:
            enabled: Whether verbose printing is enabled (default: True)
        """
        self.enabled = enabled
        self.use_rich = enabled and RICH_AVAILABLE
        self.console = Console() if self.use_rich else None
        
        if enabled and not RICH_AVAILABLE:
            print("Warning: Rich not installed, falling back to simple output")
    
    def print_header(self, query: str, context: str, max_iters: int):
        """Print RLM session header."""
        if not self.enabled:
            return
        
        if self.use_rich:
            title = Text("RLM Session", style="bold blue")
            content = f"Query: {query[:100]}...\nMax iterations: {max_iters}"
            panel = Panel(content, title=title, border_style="blue")
            self.console.print()
            self.console.print(panel)
            self.console.print()
        else:
            print("\n" + "="*70)
            print(" RLM Session")
            print("="*70)
            print(f"Query: {query[:100]}...")
            print(f"Max iterations: {max_iters}")
            print("="*70 + "\n")
    
    def print_iteration_start(self, iteration: int):
        """Print iteration separator."""
        if not self.enabled:
            return
        
        if self.use_rich:
            rule = Rule(f"Iteration {iteration}", style="cyan")
            self.console.print(rule)
        else:
            print(f"\n--- Iteration {iteration} ---")
    
    def print_response(self, response: str):
        """Print LLM response."""
        if not self.enabled:
            return
        
        if self.use_rich:
            self.console.print("[yellow]Response:[/yellow]")
            self.console.print(response)
        else:
            print("\nResponse:")
            print(response)
    
    def print_code_execution(self, code: str, result: REPLResult):
        """Print code execution details."""
        if not self.enabled:
            return
        
        if self.use_rich:
            # Code
            syntax = Syntax(code, "python", theme="monokai", line_numbers=False)
            self.console.print("\n[green]Executing:[/green]")
            self.console.print(syntax)
            
            # Output
            if result.stdout:
                self.console.print("[dim]Output:[/dim]")
                self.console.print(result.stdout)
            
            # Errors
            if result.stderr:
                self.console.print("[red]Error:[/red]")
                self.console.print(result.stderr)
            
            # Timing
            if result.execution_time:
                self.console.print(f"[dim]({result.execution_time:.3f}s)[/dim]")
        else:
            print("\nExecuting:")
            print(code)
            if result.stdout:
                print("\nOutput:")
                print(result.stdout)
            if result.stderr:
                print("\nError:")
                print(result.stderr)
            if result.execution_time:
                print(f"({result.execution_time:.3f}s)")
    
    def print_iteration(self, iteration: RLMIteration, iteration_num: int):
        """Print complete iteration."""
        if not self.enabled:
            return
        
        self.print_iteration_start(iteration_num)
        self.print_response(iteration.response)
        
        for block in iteration.code_blocks:
            self.print_code_execution(block.code, block.result)
    
    def print_final_answer(self, answer: str):
        """Print final answer."""
        if not self.enabled:
            return
        
        if self.use_rich:
            panel = Panel(
                answer,
                title=Text("Final Answer", style="bold green"),
                border_style="green"
            )
            self.console.print()
            self.console.print(panel)
            self.console.print()
        else:
            print("\n" + "="*70)
            print(" Final Answer")
            print("="*70)
            print(answer)
            print("="*70 + "\n")
    
    def print_summary(self, total_iterations: int, total_time: float):
        """Print execution summary."""
        if not self.enabled:
            return
        
        summary = f"Iterations: {total_iterations} | Time: {total_time:.2f}s"
        
        if self.use_rich:
            self.console.print()
            self.console.print(Rule(summary, style="dim"))
            self.console.print()
        else:
            print(f"\n{summary}\n")

Test VerbosePrinter:

In [ ]:
# Test with Rich if available
verbose = VerbosePrinter(enabled=True)

verbose.print_header(
    query="What is the Activity class in PROV?",
    context="PROV ontology loaded",
    max_iters=3
)

# Mock iteration
result = REPLResult(
    stdout="Activity: http://www.w3.org/ns/prov#Activity",
    stderr="",
    locals={},
    execution_time=0.05
)
block = CodeBlock(code="search_by_label(prov_meta, 'Activity')", result=result)
iteration = RLMIteration(
    prompt="Find Activity class",
    response="Let me search for the Activity class",
    code_blocks=[block]
)

verbose.print_iteration(iteration, 1)
verbose.print_final_answer("The Activity class represents processes in PROV.")
verbose.print_summary(total_iterations=1, total_time=1.5)

print("\n✓ VerbosePrinter works")